In [1]:
import pandas as pd
import numpy as np
from khaiii import KhaiiiApi
from gensim import corpora, models
from pprint import pprint


In [2]:
year = [i for i in range(2002, 2021)]
sortedresult = pd.DataFrame()

In [3]:
for i in range(19):
    print("==== "+str(year[i])+" ====")
    data = pd.read_csv("./modi_data/kor_"+str(year[i])+".csv")
    #data = data.drop(['Unnamed: 0'], axis=1)
    
    dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
    dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
    #print(dfWordList2)

    dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
    dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
    dfWordDiv = dfWordList2
    #print(dfWordMod)

    seriesDelete = dfWordDel["기존"]
    stopword = []
    for word in seriesDelete.values:
        stopword.append(word)
    #print(stopword)

    seriesModify = dfWordMod["기존"]
    modiword = []
    for word in seriesModify.values:
        modiword.append(word)
    #print(len(modiword))

    seriesModify2 = dfWordMod["수정"]
    modiword2 = []
    for word in seriesModify2.values:
        modiword2.append(word)
    #print(len(modiword2))

    seriesDivide = dfWordDiv["기존"]
    divword = []
    for word in seriesDivide.values:
        divword.append(word)
    #print(divword)
    #print(len(divword))

    seriesDivide2 = dfWordDiv["수정"]
    divword2 = []
    for words in seriesDivide2.values:
        divword2.append(words.split(', '))
    #print(divword2)
    #print(len(divword2))


    api = KhaiiiApi()
    def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
        list = []
        
        for word in api.analyze(raw): #raw data
            #print(word)
            
            for morph in word.morphs:
                #print(morph.lex)
                if len(morph.lex) > 1 and morph.tag in pos and morph.lex not in stopword: 
                    if morph.tag == 'SL':
                        morph.lex = morph.lex.lower()
                    if morph.lex in divword:
                        morph.lex = divword2[divword.index(morph.lex)]
                        list.extend(morph.lex)
                    elif morph.lex in modiword:
                        morph.lex = modiword2[modiword.index(morph.lex)]
                        list.append(morph.lex)
                    else: list.append(morph.lex)
                    
        return list

    tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
    #print(tokenized)
    #tokenized.to_excel("./finaldata/0911token_full.xls") ##############
    print("========= tokenization completed =========")

    id2word = corpora.Dictionary(tokenized)
    corpus=[id2word.doc2bow(text) for text in tokenized]
    print("# words in total : ", len(id2word))
    print("# documents : ", len(corpus))

    #tfidf
    print("==== calculating tfidf ====")
    tfidf = models.TfidfModel(corpus)

    #tfidf per doc
    tfidflist = []
    for doc in tfidf[corpus]:
        inner_list = [0]*len(id2word) 
        for id, freq in doc:
            inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
        tfidflist.append(inner_list)
    #print(len(tfidflist))

    tfidf_df = pd.DataFrame(tfidflist)
    tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
    #print(tfidf_df)

    total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
    #print(total_df)
    total_df.to_excel("./final_data/tfidf"+str(year[i])+".xlsx") 

    #sum of tfidf for each word
    columnsum = pd.DataFrame(total_df.sum(axis=0)).T
    columnsum = columnsum.drop(['no'], axis=1)
    columnsum['year'] = year[i]
    #print(columnsum)
    columnsum.to_excel("./final_data/sum"+str(year[i])+".xlsx")

    #sort tfidf value in descending order
    columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
    print(columnsum)
    columnsum.to_excel("./final_data/sorted"+str(year[i])+".xlsx")

    print("==== completed ====")

==== 2002 ====
========= tokenization completed =========
# words in total :  361
# documents :  11
==== calculating tfidf ====
   year  컴포넌트    아동   시스템    학습   판소리    모형  프로그래밍   평가   콘텐츠  ...    위쪽  \
0  2002  0.87  0.72  0.68  0.62  0.58  0.57   0.57  0.5  0.49  ...  0.07   

     변화    연습    어휘   아래쪽   아동감    실행   css    소리    상황  
0  0.07  0.07  0.07  0.07  0.07  0.07  0.07  0.07  0.07  

[1 rows x 362 columns]
==== completed ====
==== 2003 ====
========= tokenization completed =========
# words in total :  620
# documents :  26
==== calculating tfidf ====
   year    평가    상담    환경   학습    아동  컴포넌트    수행    독서    교수  ...   거부감    충족  \
0  2003  1.53  1.39  1.23  1.1  0.99  0.98  0.92  0.88  0.87  ...  0.05  0.05   

     친구    여건    원인    정신    허용    학급    편안    유리  
0  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  

[1 rows x 621 columns]
==== completed ====
==== 2004 ====
========= tokenization completed =========
# words in total :  713
# documents :  30
==== calculating tfi

In [4]:
# tfidf for full data

data = pd.read_csv("./modi_data/kor_full.csv")
#data = data.drop(['Unnamed: 0'], axis=1)
    
dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
#print(dfWordList2)

dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
dfWordDiv = dfWordList2
#print(dfWordMod)

seriesDelete = dfWordDel["기존"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["기존"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["수정"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

seriesDivide = dfWordDiv["기존"]
divword = []
for word in seriesDivide.values:
    divword.append(word)
#print(divword)
#print(len(divword))

seriesDivide2 = dfWordDiv["수정"]
divword2 = []
for words in seriesDivide2.values:
    divword2.append(words.split(', '))
#print(divword2)
#print(len(divword2))


api = KhaiiiApi()
def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    
    for word in api.analyze(raw): #raw data
        #print(word)
        
        for morph in word.morphs:
            #print(morph.lex)
            if len(morph.lex) > 1 and morph.tag in pos and morph.lex not in stopword: 
                if morph.tag == 'SL':
                    morph.lex = morph.lex.lower()
                if morph.lex in divword:
                    morph.lex = divword2[divword.index(morph.lex)]
                    list.extend(morph.lex)
                elif morph.lex in modiword:
                    morph.lex = modiword2[modiword.index(morph.lex)]
                    list.append(morph.lex)
                else: list.append(morph.lex)
                
    return list

tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
#print(tokenized)
#tokenized.to_excel("./finaldata/0911token_full.xls") ##############
print("========= tokenization completed =========")

id2word = corpora.Dictionary(tokenized)
corpus=[id2word.doc2bow(text) for text in tokenized]
print("# words in total : ", len(id2word))
print("# documents : ", len(corpus))

#tfidf
print("==== calculating tfidf ====")
tfidf = models.TfidfModel(corpus)

#tfidf per doc
tfidflist = []
for doc in tfidf[corpus]:
    inner_list = [0]*len(id2word) 
    for id, freq in doc:
        inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
    tfidflist.append(inner_list)
#print(len(tfidflist))

tfidf_df = pd.DataFrame(tfidflist)
tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
#print(tfidf_df)

total_df = pd.concat([data[["year", "no"]], tfidf_df], axis=1)
#print(total_df)
total_df.to_excel("./final_data/tfidf_full.xlsx") 

#sum of tfidf for each word
columnsum = pd.DataFrame(total_df.sum(axis=0)).T
columnsum = columnsum.drop(['no'], axis=1)
columnsum['year'] = year[i]
#print(columnsum)
columnsum.to_excel("./final_data/sum_full.xlsx")

#sort tfidf value in descending order
columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
print(columnsum)
columnsum.to_excel("./final_data/sorted_full.xlsx")

print("==== completed ====")

========= tokenization completed =========
# words in total :  4217
# documents :  812
==== calculating tfidf ====
   year  프로그래밍    로봇     정보     학습     sw    컴퓨터   컴퓨팅    교사    시스템  ...  \
0  2020  30.26  28.9  25.97  25.59  22.81  21.14  19.4  18.0  17.24  ...   

    양육자    오후    주말    오전    영유    영아    염려    주중  어머니인식    장르  
0  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04   0.04  0.04  

[1 rows x 4218 columns]
==== completed ====
